### Load Orders and Positions tables 

I created 2 csv files to represent the orders and positions tables. For the purpose of this exercise, I generated some dummy data. In the positions table, the price of wine has been fixed at 10 CHF.  The foreign key is order_id. 

To use the files, they must be imported and stored into a pandas DataFrame (similar to a SQL table). 

In [49]:
# Import pandas library used for data analysis and manipulation

import pandas as pd

In [61]:
# Load and view orders.csv file

df_orders = pd.read_csv('orders.csv')
df_orders.head()

,order_id,order_version_id,date,total,quantity,valid_from,valid_to
0,1,1,2022-01-01 00:00:00,100,10,2022-01-01 00:00:00,2022-01-05 00:00:00
1,1,2,2022-01-01 00:00:00,50,5,2022-01-05 00:00:00,9999-01-01 23:59:00
2,2,1,2022-02-01 00:00:00,100,10,2022-01-01 00:00:00,2022-02-10 00:00:00
3,2,2,2022-02-01 00:00:00,50,5,2022-02-10 00:00:00,2022-02-11 00:00:00
4,2,3,2022-02-01 00:00:00,200,20,2022-02-11 00:00:00,9999-01-01 23:59:00


In [62]:
# Load and view positions.csv file 

df_positions = pd.read_csv('positions.csv')
df_positions.head()

,order_id,position_id,wine_id,wine_name,quantity,price,netto_netto_price
0,1,1,1000,red,3,10,8
1,1,2,2000,white,2,10,9
2,2,2,2000,white,5,10,9
3,2,1,1000,red,10,10,8
4,2,3,3000,organic,5,10,7


### Show the last version of each order

#### Option 1 - using the valid_to column

In [53]:
# Since valid_to of the newest version of an order is set to a generic future datetime "9999-01-01 23:59:00", 
# we can use that value to filter the last order versions.

df_last_versions_1 = df_orders[df_orders['valid_to'] == "9999-01-01 23:59:00"] 
df_last_versions_1

,order_id,order_version_id,date,total,quantity,valid_from,valid_to
1,1,2,2022-01-01 00:00:00,50,5,2022-01-05 00:00:00,9999-01-01 23:59:00
4,2,3,2022-02-01 00:00:00,200,20,2022-02-11 00:00:00,9999-01-01 23:59:00
5,3,1,2022-03-10 00:00:00,100,10,2022-03-10 00:00:00,9999-01-01 23:59:00
8,4,3,2022-04-10 00:00:00,200,20,2022-04-12 00:00:00,9999-01-01 23:59:00
11,5,3,2022-05-02 00:00:00,250,25,2022-05-05 00:00:00,9999-01-01 23:59:00
12,6,1,2022-05-14 00:00:00,300,30,2022-05-14 00:00:00,9999-01-01 23:59:00


#### Option 2 - based on maximum order_version_id number

In [44]:
# If we assume that the ids generated for each order version are sequential numbers, 
# we can filter the table based on the highest order_version_id number of each order. 

# Because order_version_id is type string, change the type of the column to int64
df_orders.astype({'order_version_id':'int64'})

# Group orders by order_id
grouped_orders = df_orders.groupby('order_id', as_index=False)

df_last_versions_2 = grouped_orders.apply(lambda df:df.sort_values('order_version_id', ascending=False).head(1)).droplevel(0)
df_last_versions_2

,order_id,order_version_id,date,total,quantity,valid_from,valid_to
1,1,2,2022-01-01 00:00:00,50,5,2022-01-05 00:00:00,9999-01-01 23:59:00
4,2,3,2022-02-01 00:00:00,200,20,2022-02-11 00:00:00,9999-01-01 23:59:00
5,3,1,2022-03-10 00:00:00,100,10,2022-03-10 00:00:00,9999-01-01 23:59:00
8,4,3,2022-04-10 00:00:00,200,20,2022-04-12 00:00:00,9999-01-01 23:59:00
11,5,3,2022-05-02 00:00:00,250,25,2022-05-05 00:00:00,9999-01-01 23:59:00
12,6,1,2022-05-14 00:00:00,300,30,2022-05-14 00:00:00,9999-01-01 23:59:00


#### Option 2 - based on maximum valid_to date

In [47]:
# In case the valid_to date of the last order version is set to another specified date in the future, 
# instead of a fixed default value of 9999-1-1, we can find the maximum valid_to date in a table grouped
# by order_id. 

# Group orders by order_id
grouped_orders = df_orders.groupby('order_id', as_index=False)

df_last_versions_3 = grouped_orders.apply(lambda df:df.sort_values('valid_to', ascending=False).head(1)).droplevel(0)
df_last_versions_3

,order_id,order_version_id,date,total,quantity,valid_from,valid_to
1,1,2,2022-01-01 00:00:00,50,5,2022-01-05 00:00:00,9999-01-01 23:59:00
4,2,3,2022-02-01 00:00:00,200,20,2022-02-11 00:00:00,9999-01-01 23:59:00
5,3,1,2022-03-10 00:00:00,100,10,2022-03-10 00:00:00,9999-01-01 23:59:00
8,4,3,2022-04-10 00:00:00,200,20,2022-04-12 00:00:00,9999-01-01 23:59:00
11,5,3,2022-05-02 00:00:00,250,25,2022-05-05 00:00:00,9999-01-01 23:59:00
12,6,1,2022-05-14 00:00:00,300,30,2022-05-14 00:00:00,9999-01-01 23:59:00


### Show positions of the last version of each order

In [66]:
# Join tables to show the positions of the last order versions 

result = df_positions.merge(df_last_versions_1, on='order_id', how='left')
result.head()

,order_id,position_id,wine_id,wine_name,quantity_x,price,netto_netto_price,order_version_id,date,total,quantity_y,valid_from,valid_to
0,1,1,1000,red,3,10,8,2,2022-01-01 00:00:00,50,5,2022-01-05 00:00:00,9999-01-01 23:59:00
1,1,2,2000,white,2,10,9,2,2022-01-01 00:00:00,50,5,2022-01-05 00:00:00,9999-01-01 23:59:00
2,2,2,2000,white,5,10,9,3,2022-02-01 00:00:00,200,20,2022-02-11 00:00:00,9999-01-01 23:59:00
3,2,1,1000,red,10,10,8,3,2022-02-01 00:00:00,200,20,2022-02-11 00:00:00,9999-01-01 23:59:00
4,2,3,3000,organic,5,10,7,3,2022-02-01 00:00:00,200,20,2022-02-11 00:00:00,9999-01-01 23:59:00


In [77]:
# Clean the resulting view

# Rename columns that aren't straightforward and drop valid_to column, because it's irrelevant for this view
result.rename(columns = {'date':'date_of_order', 'quantity_x':'quantity', 'total': 'order_total', 
                         'quantity_y':'total_order_quantity'}, inplace=False).drop(columns = {'valid_to'}, 
                                                                                   axis=1, inplace=False)

,order_id,position_id,wine_id,wine_name,quantity,price,netto_netto_price,order_version_id,date_of_order,order_total,total_order_quantity,valid_from
0,1,1,1000,red,3,10,8,2,2022-01-01 00:00:00,50,5,2022-01-05 00:00:00
1,1,2,2000,white,2,10,9,2,2022-01-01 00:00:00,50,5,2022-01-05 00:00:00
2,2,2,2000,white,5,10,9,3,2022-02-01 00:00:00,200,20,2022-02-11 00:00:00
3,2,1,1000,red,10,10,8,3,2022-02-01 00:00:00,200,20,2022-02-11 00:00:00
4,2,3,3000,organic,5,10,7,3,2022-02-01 00:00:00,200,20,2022-02-11 00:00:00
5,3,1,2000,white,2,10,9,1,2022-03-10 00:00:00,100,10,2022-03-10 00:00:00
6,3,2,3000,organic,2,10,7,1,2022-03-10 00:00:00,100,10,2022-03-10 00:00:00
7,3,3,4000,rose,6,10,7,1,2022-03-10 00:00:00,100,10,2022-03-10 00:00:00
8,4,1,2000,white,10,10,9,3,2022-04-10 00:00:00,200,20,2022-04-12 00:00:00
9,4,2,3000,organic,10,10,7,3,2022-04-10 00:00:00,200,20,2022-04-12 00:00:00


In [11]:
# Since the task consists of creating a function, for the purpose of this exercise, 
# I will create a function to complete the first part of the code above, 
# i.e. to filter the orders table to show only the last order versions. 

# An extended function would also take the Positions file/table as an argument, 
# and join the tables for a detailed view of the orders. 

import pandas as pd
from os.path import isfile

def show_last_order_versions(orders_file):
        
    try:    
        df_orders = pd.read_csv(orders_file)
        df_last_versions = df_orders[df_orders['valid_to'] == "9999-01-01 23:59:00"] 
        return df_last_versions
    except FileNotFoundError:
        print("File not found.")  

In [12]:
show_last_order_versions("orders.csv")

,order_id,order_version_id,date,total,quantity,valid_from,valid_to
1,1,2,2022-01-01 00:00:00,50,5,2022-01-05 00:00:00,9999-01-01 23:59:00
4,2,3,2022-02-01 00:00:00,200,20,2022-02-11 00:00:00,9999-01-01 23:59:00
5,3,1,2022-03-10 00:00:00,100,10,2022-03-10 00:00:00,9999-01-01 23:59:00
8,4,3,2022-04-10 00:00:00,200,20,2022-04-12 00:00:00,9999-01-01 23:59:00
11,5,3,2022-05-02 00:00:00,250,25,2022-05-05 00:00:00,9999-01-01 23:59:00
12,6,1,2022-05-14 00:00:00,300,30,2022-05-14 00:00:00,9999-01-01 23:59:00
